### Task 1: FASHION MNIST CLASSIFICATION WITH CNN

In [ ]:
import torch
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Download the training dataset
train_dataset = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=transform
)

# Download the test dataset
test_dataset = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=transform
)

print("Fashion-MNIST dataset downloaded successfully!")


100%|██████████| 26421880/26421880 [00:02<00:00, 10204735.52it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 194182.87it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:06<00:00, 664653.75it/s] 


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 15467247.13it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Fashion-MNIST dataset downloaded successfully!


Checking whether GPU is available or not

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Checking Particular elements of training set

In [ ]:
# Get the first sample from the training dataset
image, label = train_dataset[0]

# Print the shape and values of the image tensor
print(f"Shape of the image tensor: {image.shape}")
print("Image tensor values:\n", image)

# Print the label of the image
print(f"\nLabel: {label}")
# Get the first sample from the training dataset
image, label = train_dataset[0]

# Check if the image is a 3D tensor
is_3d = len(image.shape) == 3

print(f"Is the image a 3D tensor? {is_3d}")
print(f"Shape of the image tensor: {image.shape}")



Shape of the image tensor: torch.Size([1, 28, 28])
Image tensor values:
 tensor([[[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1

Check the type and structure of elements in train_dataset

In [ ]:
print(type(train_dataset[0]))
print(f"Image type: {type(train_dataset[0][0])}")
print(f"Label type: {type(train_dataset[0][1])}")


<class 'tuple'>
Image type: <class 'torch.Tensor'>
Label type: <class 'int'>


Relabeling the images as per the requirements

In [ ]:
# Define the mapping for new labels
def relabel(label):
    # Clothes: 0
    if label in [0, 1, 2, 3, 4, 6]:
        return 0
    # Shoes: 1
    elif label in [5, 7, 9]:
        return 1
    # Others: 2
    elif label == 8:
        return 2

train_dataset_relabel = [(image, relabel(label)) for image, label in train_dataset]
test_dataset_relabel = [(image, relabel(label)) for image, label in test_dataset]

# Check the first few relabeled samples
for i in range(5):
    image, new_label = train_dataset_relabel[i]
    print(f"Original Label: {train_dataset[i][1]}, New Label: {new_label}")


Original Label: 9, New Label: 1
Original Label: 0, New Label: 0
Original Label: 0, New Label: 0
Original Label: 3, New Label: 0
Original Label: 0, New Label: 0


Constructing the CNN Model to be trained

In [ ]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(CNN, self).__init__()
        self.input_size = input_size
        self.output_size = output_size

        self.network = nn.Sequential(
            # Layer 0: Convolutional Layer with kernel size 5
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # Pooling layer to reduce spatial size

            # Layer 1: Convolutional Layer with kernel size 3
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # Pooling layer to reduce spatial size

            # Layer 2: Convolutional Layer with kernel size 3
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),  # Pooling layer to reduce spatial size

            # Layer 3: Convolutional Layer with kernel size 1 (1x1 convolution)
            nn.Conv2d(128, 256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            # Dropout layer for regularization
            nn.Dropout(0.4),

            # Layer 4: Global Average Pooling
            nn.AdaptiveAvgPool2d(1),  # Reduces each channel to a single value
            nn.Flatten(),

            # Fully Connected Layer
            nn.Linear(256, output_size),  # Final layer with the number of classes
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        return self.network(x)

# Create the new custom CNN model instance
input_size = 1  # Single channel (grayscale) images
output_size = 3  # Three categories: Clothes, Shoes, Others
model = CNN(input_size, output_size)

# Print the new model architecture
print(model)


CNN(
  (network): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (

Training the model with the relabelled training set

In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

# Check if CUDA is available and use it if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move the model to the device

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create DataLoader objects for training and test sets
batch_size = 64
train_loader = DataLoader(train_dataset_relabel, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset_relabel, batch_size=batch_size, shuffle=False)

# Training the model
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to the device

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print epoch statistics
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")

print("Training complete!")


Epoch 1/10, Loss: 0.0370
Epoch 2/10, Loss: 0.0148
Epoch 3/10, Loss: 0.0109
Epoch 4/10, Loss: 0.0082
Epoch 5/10, Loss: 0.0064
Epoch 6/10, Loss: 0.0057
Epoch 7/10, Loss: 0.0049
Epoch 8/10, Loss: 0.0038
Epoch 9/10, Loss: 0.0042
Epoch 10/10, Loss: 0.0024
Training complete!


Checking the model accuracy on the relabelled test set

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():  # Disable gradient computation for evaluation
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)  # Forward pass
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate and print the accuracy
accuracy = 100 * correct / total
print(f"Accuracy on the test set: {accuracy:.2f}%")


Accuracy on the test set: 99.65%


In [ ]:
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import DataLoader

Permuting the training and test data set with a fixed permutation

In [ ]:
import torch
from torch.utils.data import DataLoader
import numpy as np
from torchvision import datasets, transforms

# Create a permutation index for 28x28 pixels
permute_idx = np.random.permutation(28 * 28)

# Define a custom transform that permutes pixels
class PermuteTransform:
    def __init__(self, permute_idx):
        self.permute_idx = permute_idx

    def __call__(self, image):
        image = image.view(-1)
        image = image[self.permute_idx]
        return image.view(1, 28, 28)

# Create a permuted version of the dataset
permuted_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    PermuteTransform(permute_idx)
])

# Create permuted train and test datasets
train_dataset_permuted = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=permuted_transform
)

test_dataset_permuted = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=permuted_transform
)

# Define the batch size
batch_size = 64




Again Relabelling the training and the test set as per the requirements

In [ ]:
def relabel(label):
    if label in [0, 1, 2, 3, 4, 6]:
        return 0  # Clothes
    elif label in [5, 7, 9]:
        return 1  # Shoes
    elif label == 8:
        return 2  # Others

# Relabel the permuted train and test datasets
train_dataset_permuted_relabel = [(image, relabel(label)) for image, label in train_dataset_permuted]
test_dataset_permuted_relabel = [(image, relabel(label)) for image, label in test_dataset_permuted]

# Define the batch size
batch_size = 64


# Create DataLoader for relabeled permuted datasets
train_loader_permuted_relabel = DataLoader(train_dataset_permuted_relabel, batch_size=batch_size, shuffle=True)
test_loader_permuted_relabel = DataLoader(test_dataset_permuted_relabel, batch_size=batch_size, shuffle=False)


Training the model with the permuted and relabelled training set

In [ ]:
# Re-initialize the model for training on relabeled permuted data
model_permuted_relabel = CNN(input_size, output_size)
model_permuted_relabel.to(device)

# Define loss function and optimizer
optimizer_permuted_relabel = optim.Adam(model_permuted_relabel.parameters(), lr=0.001)

# Train the model on relabeled permuted dataset
for epoch in range(epochs):
    model_permuted_relabel.train()
    running_loss = 0.0

    for images, labels in train_loader_permuted_relabel:
        images, labels = images.to(device), labels.to(device)

        optimizer_permuted_relabel.zero_grad()
        outputs = model_permuted_relabel(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_permuted_relabel.step()

        running_loss += loss.item()

    print(f"Permuted Relabeled Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader_permuted_relabel):.4f}")

print("Training on relabeled permuted data complete!")


Permuted Relabeled Epoch 1/10, Loss: 0.0616
Permuted Relabeled Epoch 2/10, Loss: 0.0264
Permuted Relabeled Epoch 3/10, Loss: 0.0169
Permuted Relabeled Epoch 4/10, Loss: 0.0128
Permuted Relabeled Epoch 5/10, Loss: 0.0097
Permuted Relabeled Epoch 6/10, Loss: 0.0076
Permuted Relabeled Epoch 7/10, Loss: 0.0064
Permuted Relabeled Epoch 8/10, Loss: 0.0051
Permuted Relabeled Epoch 9/10, Loss: 0.0045
Permuted Relabeled Epoch 10/10, Loss: 0.0043
Training on relabeled permuted data complete!


Checking the classification accuracy on the relabelled training and the test set

In [ ]:
# Evaluate the model on the relabeled permuted test set
model_permuted_relabel.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader_permuted_relabel:
        images, labels = images.to(device), labels.to(device)
        outputs = model_permuted_relabel(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy_permuted_relabel = 100 * correct / total
print(f"Accuracy on the relabeled permuted test set: {accuracy_permuted_relabel:.2f}%")


Accuracy on the relabeled permuted test set: 98.99%


### Task 2: RESNET for Emotion detection

The required data is uploading


In [ ]:
from google.colab import files
files.upload()


Saving emotion_detection_dataset.zip to emotion_detection_dataset.zip


unzipping it

In [ ]:
!unzip -q emotion_detection_dataset.zip


In [ ]:
import os


os.listdir()


['.config',
 'emotion_detection_dataset',
 'emotion_detection_dataset.zip',
 'sample_data']

In [ ]:
import os

# List the contents of the current directory
os.listdir()


['.config',
 'emotion_detection_dataset',
 'emotion_detection_dataset.zip',
 'sample_data']

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


Transforming the images in a size as the requirements and normalizing it

In [ ]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
# Paths to the train and test directories
train_dir = './emotion_detection_dataset/train'
test_dir = './emotion_detection_dataset/test'

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)


In [ ]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

In [ ]:
from torchvision import models
import torch.nn as nn


Initializing a Resnet18 network to train from scratch

In [ ]:
# Initialize the ResNet-18 model
model = models.resnet18(pretrained=False)

num_features = model.fc.in_features
num_classes = len(train_dataset.classes)  # Number of classes in your dataset
model.fc = nn.Linear(num_features, num_classes)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
print(num_classes)

7


In [ ]:
len(train_dataset)

28709

In [ ]:
len(test_dataset)

7178

In [ ]:
import torch
if torch.cuda.is_available():
    print("GPU is available and ready to use!")
else:
    print("GPU not available, using CPU.")


GPU is available and ready to use!


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.001)


Training the untrained Resnet18 model

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

print("Training complete!")


Epoch [1/10], Loss: 1.7590
Epoch [2/10], Loss: 1.3933
Epoch [3/10], Loss: 1.2228
Epoch [4/10], Loss: 1.1161
Epoch [5/10], Loss: 1.0285
Epoch [6/10], Loss: 0.9385
Epoch [7/10], Loss: 0.8282
Epoch [8/10], Loss: 0.6951
Epoch [9/10], Loss: 0.5273
Epoch [10/10], Loss: 0.3726
Training complete!


Checking performabce on the test set

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')


Accuracy on the test set: 60.14%


Now fine tuning a pretrained Resnet18 classifier which is in built in pytorch

In [ ]:
# Load pre-trained ResNet-18 model
model_pretrained = models.resnet18(pretrained=True)

for param in model_pretrained.parameters():
    param.requires_grad = False

# Modify the final fully connected layer
num_features = model_pretrained.fc.in_features
model_pretrained.fc = nn.Linear(num_features, num_classes)  # Adjusting for the number of classes in the dataset

model_pretrained = model_pretrained.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 185MB/s]


In [ ]:
optimizer_pretrained = optim.Adam(model_pretrained.fc.parameters(), lr=0.001)


Training on the training set

In [ ]:
num_epochs = 10  # You can adjust the number of epochs as needed

for epoch in range(num_epochs):
    model_pretrained.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer_pretrained.zero_grad()  # Zero the parameter gradients
        outputs = model_pretrained(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_pretrained.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

print("Fine-tuning complete!")


Epoch [1/10], Loss: 1.5947
Epoch [2/10], Loss: 1.5002
Epoch [3/10], Loss: 1.4699
Epoch [4/10], Loss: 1.4537
Epoch [5/10], Loss: 1.4508
Epoch [6/10], Loss: 1.4460
Epoch [7/10], Loss: 1.4426
Epoch [8/10], Loss: 1.4393
Epoch [9/10], Loss: 1.4330
Epoch [10/10], Loss: 1.4319
Fine-tuning complete!


Checking it's accuracy on the test set

In [ ]:
model_pretrained.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_pretrained(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the fine-tuned model on the test set: {accuracy:.2f}%')


Accuracy of the fine-tuned model on the test set: 45.61%
